In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql import types
import pyspark.sql.functions as F #(func)
from pyspark.sql.functions import col
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
print("Welcome to the W261 final project!") 


# Know your mount
Here is the mounting for this class, your source for the original data! Remember, you only have Read access, not Write! Also, become familiar with `dbutils` the equivalent of `gcp` in DataProc

In [0]:
data_BASE_DIR = "dbfs:/mnt/mids-w261/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/"))

In [0]:
dbutils.fs.help()

# Data for the Project

For the project you will have 4 sources of data:

1. Airlines Data: This is the raw data of flights information. You have 3 months, 6 months, 1 year, and full data from 2015 to 2019. Remember the maxima: "Test, Test, Test", so a lot of testing in smaller samples before scaling up! Location of the data? `dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/`, `dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/`, etc. (Below the dbutils to get the folders)
2. Weather Data: Raw data for weather information. Same as before, we are sharing 3 months, 6 months, 1 year
3. Stations data: Extra information of the location of the different weather stations. Location `dbfs:/mnt/mids-w261/datasets_final_project_2022/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/`
4. OTPW Data: This is our joined data (We joined Airlines and Weather). This is the main dataset for your project, the previous 3 are given for reference. You can attempt your own join for Extra Credit. Location `dbfs:/mnt/mids-w261/OTPW_60M/OTPW_60M/` and more, several samples are given!

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/student-groups/"))

# display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/"))      # individual datasets by time increments

display(dbutils.fs.ls("dbfs:/mnt/mids-w261/OTPW_1D_CSV/OTPW_1D_CSV/"))      # combined datasets

display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/"))      # broken out by each year
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2015/"))      
# display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/"))

display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/"))      # broken out by each year
# display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m/"))

# display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/"))      # one dataset

In [0]:
def get_dir_size_gb(path):
    files = dbutils.fs.ls(path)
    total_bytes = sum([f.size for f in files])
    return total_bytes / (1024 ** 3)


In [0]:
# Airline Data    
df_flights_5y = spark.read.parquet(
    f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2015/",
    f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2016/",
    f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2017/",
    f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2018/",
    f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2019/"
)
print(f"Approximate size in GB: {get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2015/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2016/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2017/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2018/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/YEAR=2019/')}")
display(df_flights_5y)

In [0]:
# Weather data
df_weather_5y = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2015/",
                                   f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2016/",
                                   f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2017/",
                                   f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2018",
                                   f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2019/")
print(f"Approximate size in GB: {get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2015/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2016/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2017/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2018/') + get_dir_size_gb('dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/YEAR=2019/')}")
print(df_weather_5y.count(), len(df_weather_5y.columns))
display(df_weather_5y)

In [0]:
# Stations data      
df_stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/")
print(f"Approximate size in GB: {get_dir_size_gb("dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/")}")
display(df_stations)

In [0]:
# OTPW
df_otpw = spark.read.format("csv").option("header","true").load(f"dbfs:/mnt/mids-w261/OTPW_3M_2015.csv")
print(f"Approximate size in GB: {get_dir_size_gb("dbfs:/mnt/mids-w261/OTPW_3M_2015.csv")}")
display(df_otpw)

In [0]:
# Airport Codes
df_airports = spark.read.format("csv").option("header","true").load(f"dbfs:/mnt/mids-w261/airport-codes_csv.csv")
print(f"Approximate size in GB: {get_dir_size_gb("dbfs:/mnt/mids-w261/airport-codes_csv.csv")}")
display(df_airports)


# EDA

In [0]:
# read as csv, write as parquet, read back as parquet to do analysis

# %pip install --upgrade matplotlib

## Airports EDA

In [0]:
df_airports = df_airports.cache()

print(f"Number of rows: {df_airports.count()}")
print(f"Number of columns: {len(df_airports.columns)}")

display(df_airports.limit(10))

display(df_airports.describe())

# convert elevation_ft to float
df_airports = df_airports.withColumn("elevation_ft", df_airports["elevation_ft"].cast("float")) 

sum_stats = [
    (c, 
     df_airports.select(c).count(),
     df_airports.select(c).distinct().count(),
     df_airports.filter(F.col(c).isNull()).count(),
     df_airports.agg(F.mean(c)).collect()[0][0],
     df_airports.agg(F.stddev(c)).collect()[0][0],
     df_airports.agg(F.min(c)).collect()[0][0],
     df_airports.agg(F.max(c)).collect()[0][0]
     ) for c in df_airports.columns
]
display(
    spark.createDataFrame(
        sum_stats,
        ["column", "count", "unique_count", "null_count", "mean", "stddev", "min", "max"]
    )
)

In [0]:

# count of nulls, visualized
columns = [stat[0] for stat in sum_stats]
null_counts = [stat[3] for stat in sum_stats]
sorted_data = sorted(zip(columns, null_counts), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_null_counts = zip(*sorted_data)
plt.figure(figsize=(12, 6))
plt.bar(sorted_columns, sorted_null_counts)
plt.title("Number of Nulls per Feature in df_airports")
plt.xlabel("Features")
plt.ylabel("Null Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# different types of airports
type_counts = df_airports.groupBy("type").count().collect()
display(type_counts)
types = [row['type'] for row in type_counts]
counts = [row['count'] for row in type_counts]
sorted_data = sorted(zip(types, counts), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_counts = zip(*sorted_data)
plt.figure(figsize=(12, 6))
plt.bar(sorted_columns, sorted_counts)
plt.title("Counts per Airport Type in df_airports")
plt.xlabel("Airport Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Flights EDA

In [0]:
df_flights_5y = df_flights_5y.dropDuplicates().cache()

print(f"Number of rows: {df_flights_5y.count()}")
print(f"Number of columns: {len(df_flights_5y.columns)}")

# display(df_flights_1y.limit(10))

df_flights_5y = df_flights_5y.withColumn(
    "FL_ID",
    F.concat_ws("_",
        F.col("FL_DATE").cast("string"),
        F.col("OP_CARRIER").cast("string"),
        F.col("OP_CARRIER_FL_NUM").cast("string"),
        F.col("ORIGIN_AIRPORT_ID").cast("string")
    )
)

df_flights_5y = df_flights_5y.withColumn("FL_ID_HASH", F.hash(F.col("FL_ID")))

display(df_flights_5y.limit(10))
display(df_flights_5y.describe())

agg_exprs = []
for c in df_flights_5y.columns:
    agg_exprs.extend([
        F.count(c).alias(f"{c}_count"),
        F.countDistinct(c).alias(f"{c}_distinct"),
        F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"{c}_nulls")
    ])
stats_row = df_flights_5y.agg(*agg_exprs).collect()[0]
sum_stats_1y = [
    (c, stats_row[f"{c}_count"], stats_row[f"{c}_distinct"], stats_row[f"{c}_nulls"])
    for c in df_flights_5y.columns
]
display(
    spark.createDataFrame(
        sum_stats_1y,
        ["column", "non_null_count", "unique_count", "null_count"]
    )
)


columns = [stat[0] for stat in sum_stats_1y]
# null_counts = [stat[3] for stat in sum_stats]
null_percentages = [(stat[3] / (stat[1]+stat[3])) * 100 for stat in sum_stats_1y]
sorted_data = sorted(zip(columns, null_percentages), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_null_percentages = zip(*sorted_data)
plt.figure(figsize=(12, 20))  # Taller figure for horizontal bars
plt.barh(sorted_columns, sorted_null_percentages)
plt.title("% Nulls per Feature in df_flights", fontsize=14, fontweight='bold')
plt.xlabel("Null %", fontsize=12)
plt.ylabel("Features", fontsize=12)
plt.tight_layout()
plt.show()

display(df_flights_5y.groupBy("OP_CARRIER").count().orderBy("count", ascending=False))
# time series of flights per day, maybe split out by carrier
carrier_flights = df_flights_5y.groupBy(["FL_DATE", "OP_CARRIER"]).count().orderBy("FL_DATE").toPandas()
display(carrier_flights)
carrier_flights['FL_DATE'] = pd.to_datetime(carrier_flights['FL_DATE'])
carrier_pivot = carrier_flights.pivot(index='FL_DATE', columns='OP_CARRIER', values='count').fillna(0)
carrier_totals = carrier_pivot.sum().sort_values(ascending=False)
carrier_pivot = carrier_pivot[carrier_totals.index]
n_carriers = len(carrier_pivot.columns)
colors = plt.cm.tab20(np.linspace(0, 1, n_carriers))
plt.figure(figsize=(16, 8))
plt.stackplot(carrier_pivot.index, 
              *[carrier_pivot[col] for col in carrier_pivot.columns],
              labels=carrier_pivot.columns,
              colors=colors,
              alpha=0.8)
plt.title("Daily Flight Counts by Carrier (Stacked by Volume)", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Number of Flights", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



display(df_flights_5y.groupBy("DEP_DEL15").count())
display(df_flights_5y.groupBy("ARR_DEL15").count())


In [0]:
display(df_airports.filter((F.col("type") == "large_airport") & (F.col("iso_country") == "US")))
df_us_airports = df_airports.filter((F.col("iso_country") == "US"))

# join df_flights and df_airports on ORIGIN and local_code
df_flights_5y_joined = df_flights_5y.join(df_us_airports, df_flights_5y.ORIGIN == df_us_airports.local_code, "inner")
display(df_flights_5y_joined.limit(10))

# visualize number of flights by TYPE (airport type) and DELAYED (DEP_DEL15)
display(
    df_flights_5y_joined.groupBy("type", "DEP_DEL15").count().orderBy("type", "DEP_DEL15")
)
airport_types_delay = df_flights_5y_joined.groupBy("type", "DEP_DEL15").count().orderBy("type", "DEP_DEL15").toPandas()
plt.figure(figsize=(12, 6))
for delayed, color in zip([0, 1], ['skyblue', 'orange']):
    subset = airport_types_delay[airport_types_delay['DEP_DEL15'] == delayed]
    plt.bar(subset['type'], subset['count'], label=f'Delayed={delayed}', color=color, alpha=0.7)
plt.title("Number of Flights by Airport Type and Delay Status (1Y)")
plt.xlabel("Airport Type")
plt.ylabel("Number of Flights")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# visualize number of flights by airport
display(df_flights_5y_joined.groupBy("ORIGIN").count().orderBy("count", ascending=False))
airport_counts = df_flights_5y_joined.groupBy("ORIGIN").count().orderBy("count", ascending=False).toPandas()
# side by side of all airports and top 10 airports
plt.figure(figsize=(18, 6))
# All airports
plt.subplot(1, 2, 1)
plt.bar(airport_counts['ORIGIN'], airport_counts['count'], color='skyblue')
plt.title("Number of Flights by Airport (1Y) - All Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Flights")
plt.xticks(rotation=90)
plt.tight_layout()
# Top 10 airports
plt.subplot(1, 2, 2)
top10 = airport_counts.head(10)
plt.bar(top10['ORIGIN'], top10['count'], color='orange')
plt.title("Number of Flights by Airport (1Y) - Top 10 Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Flights")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Weather EDA

In [0]:
us_stations = df_stations \
    .filter("distance_to_neighbor == 0") \
    .select(["station_id", "neighbor_name", "neighbor_state"])

joined_test = df_weather_5y.join(us_stations, df_weather_5y.STATION==us_stations.station_id, how='inner').drop("station_id", "neighbor_name", "neighbor_state")
df_weather_5y = joined_test.cache()

print(f"Number of rows: {df_weather_5y.count()}")
print(f"Number of columns: {len(df_weather_5y.columns)}")

display(df_weather_5y.limit(10))

display(df_weather_5y.describe())

agg_exprs = []
for c in df_weather_5y.columns:
    agg_exprs.extend([
        F.count(c).alias(f"{c}_count"),
        F.countDistinct(c).alias(f"{c}_distinct"),
        F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"{c}_nulls")
    ])
stats_row = df_weather_5y.agg(*agg_exprs).collect()[0]
sum_stats = [
    (c, stats_row[f"{c}_count"], stats_row[f"{c}_distinct"], stats_row[f"{c}_nulls"])
    for c in df_weather_5y.columns
]
display(
    spark.createDataFrame(
        sum_stats,
        ["column", "non_null_count", "unique_count", "null_count"]
    )
)

In [0]:
columns = [stat[0] for stat in sum_stats]
# null_counts = [stat[3] for stat in sum_stats]
null_percentages = [(stat[3] / (stat[1]+stat[3])) * 100 for stat in sum_stats]
sorted_data = sorted(zip(columns, null_percentages), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_null_percentages = zip(*sorted_data)
plt.figure(figsize=(12, 20))  # Taller figure for horizontal bars
plt.barh(sorted_columns, sorted_null_percentages)
plt.title("% Nulls per Feature in df_weather", fontsize=14, fontweight='bold')
plt.xlabel("Null %", fontsize=12)
plt.ylabel("Features", fontsize=12)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 10))
null_pcts = [(stat[3] / (stat[1]+stat[3])) * 100 for stat in sum_stats]
plt.hist(null_pcts, bins=30, edgecolor='black', color='skyblue')
plt.title("Distribution of Null Percentages Across All Features", fontsize=12, fontweight='bold')
plt.xlabel("Null Percentage (%)")
plt.ylabel("Number of Features")
plt.axvline(x=np.median(null_pcts), color='red', linestyle='--', label=f'Median: {np.median(null_pcts):.1f}%')
plt.legend()
plt.tight_layout()
plt.show()
print(f"Features with >50% nulls: {sum(1 for p in null_pcts if p > 50)}")
print(f"Features with 0% nulls: {sum(1 for p in null_pcts if p == 0)}")


# time series of weather reports per day ----- we should just look at the weather report for each hour, that's where the spikes are and when we have the most data
weather_reports = df_weather_5y.groupBy("DATE").count().orderBy("DATE").toPandas()
display(weather_reports)
weather_reports['DATE'] = pd.to_datetime(weather_reports['DATE'])
plt.figure(figsize=(16, 8))
plt.plot(weather_reports['DATE'], weather_reports['count'], linewidth=2, color='steelblue')
plt.title("Daily Weather Reports Over Time", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Number of Weather Reports", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
# Optional: Print some summary stats
print(f"Total days: {len(weather_reports)}")
print(f"Average reports per day: {weather_reports['count'].mean():.2f}")
print(f"Max reports in a day: {weather_reports['count'].max()}")
print(f"Min reports in a day: {weather_reports['count'].min()}")


# station, count number of distinct weather reports (dates)
num_reports = df_weather_5y.groupBy("STATION").agg(F.countDistinct("DATE").alias("DATE")).toPandas()
display(num_reports)
plt.figure(figsize=(12, 6))
plt.hist(num_reports['DATE'], bins=50, edgecolor='black', color='skyblue')
plt.title("Distribution of Number of Weather Reports per Station", fontsize=14, fontweight='bold')
plt.xlabel("Number of Weather Reports", fontsize=12)
plt.ylabel("Number of Stations", fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Stations EDA

In [0]:
df_stations = df_stations.cache()

print(f"Number of rows: {df_stations.count()}")
print(f"Number of columns: {len(df_stations.columns)}")

display(df_stations.limit(10))

display(df_stations.describe())

sum_stats = [
    (c, 
     df_stations.select(c).count(),
     df_stations.select(c).distinct().count(),
     df_stations.filter(F.col(c).isNull()).count(),
     df_stations.agg(F.mean(c)).collect()[0][0],
     df_stations.agg(F.stddev(c)).collect()[0][0],
     df_stations.agg(F.min(c)).collect()[0][0],
     df_stations.agg(F.max(c)).collect()[0][0]
     ) for c in df_stations.columns
]
display(
    spark.createDataFrame(
        sum_stats,
        ["column", "count", "unique_count", "null_count", "mean", "stddev", "min", "max"]
    )
)

columns = [stat[0] for stat in sum_stats]
null_counts = [stat[3] for stat in sum_stats]
# null_percentages = [(stat[3] / stat[1]) * 100 for stat in sum_stats]
sorted_data = sorted(zip(columns, null_counts), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_null_counts = zip(*sorted_data)
plt.figure(figsize=(12, 6))
plt.bar(sorted_columns, sorted_null_counts)
plt.title("Number of Nulls per Feature in df_stations")
plt.ylabel("Null Count")
plt.xlabel("Features")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# station_id, count number of distinct neighbors
num_neighbors = df_stations.groupBy("station_id").agg(F.countDistinct("neighbor_id").alias("num_neighbors")).toPandas()
display(num_neighbors)
plt.figure(figsize=(12, 6))
plt.hist(num_neighbors['num_neighbors'], bins=50, edgecolor='black', color='skyblue')
plt.title("Distribution of Number of Neighbors per Station", fontsize=14, fontweight='bold')
plt.xlabel("Number of Neighbors", fontsize=12)
plt.ylabel("Number of Stations", fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Custom Joined Data EDA

In [0]:
# paths
custom_join_5y_path ='dbfs:/student-groups/Group_2_2/5_year_custom_joined'


join_data_5y = spark.read.parquet(custom_join_5y_path)

# drop null flight_uid
join_data_5y_df = join_data_5y.dropna(subset=['flight_uid'])
display(join_data_5y_df)

# check 1y dataset
print(f'rows: {join_data_5y_df.count()}')
print(f'cols: {len(join_data_5y_df.columns)}\n')

join_data_5y_df.printSchema()

# df = df.filter(F.col("CANCELLED") != 1)
# print(df.count())
# display(df.limit(10))

In [0]:
join_data_5y_df = join_data_5y_df.dropDuplicates().cache()

join_data_5y_df = join_data_5y_df.withColumn(
    "FL_ID",
    F.concat_ws("_",
        F.col("FL_DATE").cast("string"),
        F.col("OP_CARRIER").cast("string"),
        F.col("OP_CARRIER_FL_NUM").cast("string"),
        F.col("ORIGIN_AIRPORT_ID").cast("string")
    )
)

join_data_5y_df = join_data_5y_df.withColumn("FL_ID_HASH", F.hash(F.col("FL_ID")))

display(join_data_5y_df.limit(10))
display(join_data_5y_df.describe())

agg_exprs = []
for c in join_data_5y_df.columns:
    agg_exprs.extend([
        F.count(c).alias(f"{c}_count"),
        F.countDistinct(c).alias(f"{c}_distinct"),
        F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"{c}_nulls")
    ])
stats_row = join_data_5y_df.agg(*agg_exprs).collect()[0]
sum_stats_1y = [
    (c, stats_row[f"{c}_count"], stats_row[f"{c}_distinct"], stats_row[f"{c}_nulls"])
    for c in join_data_5y_df.columns
]
display(
    spark.createDataFrame(
        sum_stats_1y,
        ["column", "non_null_count", "unique_count", "null_count"]
    )
)


In [0]:

columns = [stat[0] for stat in sum_stats_1y]
# null_counts = [stat[3] for stat in sum_stats]
null_percentages = [(stat[3] / (stat[1]+stat[3])) * 100 for stat in sum_stats_1y]
sorted_data = sorted(zip(columns, null_percentages), key=lambda x: x[1], reverse=True)
sorted_columns, sorted_null_percentages = zip(*sorted_data)
plt.figure(figsize=(12, 20))  # Taller figure for horizontal bars
plt.barh(sorted_columns, sorted_null_percentages)
plt.title("% Nulls per Feature in df_flights", fontsize=14, fontweight='bold')
plt.xlabel("Null %", fontsize=12)
plt.ylabel("Features", fontsize=12)
plt.tight_layout()
plt.show()

display(join_data_5y_df.groupBy("OP_CARRIER").count().orderBy("count", ascending=False))
# time series of flights per day, maybe split out by carrier
carrier_flights = join_data_5y_df.groupBy(["FL_DATE", "OP_CARRIER"]).count().orderBy("FL_DATE").toPandas()
display(carrier_flights)
carrier_flights['FL_DATE'] = pd.to_datetime(carrier_flights['FL_DATE'])
carrier_pivot = carrier_flights.pivot(index='FL_DATE', columns='OP_CARRIER', values='count').fillna(0)
carrier_totals = carrier_pivot.sum().sort_values(ascending=False)
carrier_pivot = carrier_pivot[carrier_totals.index]
n_carriers = len(carrier_pivot.columns)
colors = plt.cm.tab20(np.linspace(0, 1, n_carriers))
plt.figure(figsize=(16, 8))
plt.stackplot(carrier_pivot.index, 
              *[carrier_pivot[col] for col in carrier_pivot.columns],
              labels=carrier_pivot.columns,
              colors=colors,
              alpha=0.8)
plt.title("Daily Flight Counts by Carrier (Stacked by Volume)", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Number of Flights", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



display(join_data_5y_df.groupBy("DEP_DEL15").count())
display(join_data_5y_df.groupBy("ARR_DEL15").count())

# number of flights by airport
display(join_data_5y_df.groupBy("ORIGIN").count().orderBy("count", ascending=False))
airport_counts = join_data_5y_df.groupBy("ORIGIN").count().orderBy("count", ascending=False).toPandas()
# side by side of all airports and top 10 airports
plt.figure(figsize=(18, 6))
# All airports
plt.subplot(1, 2, 1)
plt.bar(airport_counts['ORIGIN'], airport_counts['count'], color='skyblue')
plt.title("Number of Flights by Airport - All Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Flights")
plt.xticks(rotation=90)
plt.tight_layout()
# Top 10 airports
plt.subplot(1, 2, 2)
top10 = airport_counts.head(10)
plt.bar(top10['ORIGIN'], top10['count'], color='orange')
plt.title("Number of Flights by Airport - Top 10 Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Flights")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# number of flight delays by airport
delayed_by_airport = join_data_5y_df.filter(F.col("DEP_DEL15") == 1).groupBy("ORIGIN").count().orderBy("count", ascending=False)
display(delayed_by_airport)
delayed_airport_counts = delayed_by_airport.toPandas()
plt.figure(figsize=(18, 6))
# All airports
plt.subplot(1, 2, 1)
plt.bar(delayed_airport_counts['ORIGIN'], delayed_airport_counts['count'], color='salmon')
plt.title("Number of Delayed Flights by Airport - All Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Delayed Flights")
plt.xticks(rotation=90)
plt.tight_layout()
# Top 10 airports
plt.subplot(1, 2, 2)
top10_delayed = delayed_airport_counts.head(10)
plt.bar(top10_delayed['ORIGIN'], top10_delayed['count'], color='orange')
plt.title("Number of Delayed Flights by Airport - Top 10 Airports")
plt.xlabel("Airport")
plt.ylabel("Number of Delayed Flights")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# # time series of weather reports per day ----- we should just look at the weather report for each hour, that's where the spikes are and when we have the most data
# weather_reports = join_data_1y_df.groupBy("DATE").count().orderBy("DATE").toPandas()
# display(weather_reports)
# weather_reports['DATE'] = pd.to_datetime(weather_reports['DATE'])
# plt.figure(figsize=(16, 8))
# plt.plot(weather_reports['DATE'], weather_reports['count'], linewidth=2, color='steelblue')
# plt.title("Daily Weather Reports Over Time", fontsize=14, fontweight='bold')
# plt.xlabel("Date", fontsize=12)
# plt.ylabel("Number of Weather Reports", fontsize=12)
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.show()
# # Optional: Print some summary stats
# print(f"Total days: {len(weather_reports)}")
# print(f"Average reports per day: {weather_reports['count'].mean():.2f}")
# print(f"Max reports in a day: {weather_reports['count'].max()}")
# print(f"Min reports in a day: {weather_reports['count'].min()}")


# # station, count number of distinct weather reports (dates)
# num_reports = df_weather_1y.groupBy("STATION").agg(F.countDistinct("DATE").alias("DATE")).toPandas()
# display(num_reports)
# plt.figure(figsize=(12, 6))
# plt.hist(num_reports['DATE'], bins=50, edgecolor='black', color='skyblue')
# plt.title("Distribution of Number of Weather Reports per Station", fontsize=14, fontweight='bold')
# plt.xlabel("Number of Weather Reports", fontsize=12)
# plt.ylabel("Number of Stations", fontsize=12)
# plt.grid(True, alpha=0.3, axis='y')
# plt.tight_layout()
# plt.show()

In [0]:
# for delayed flights, plotting a histogram of the number of flights at each time of day
delayed_flights = join_data_5y_df.filter(F.col("DEP_DEL15") == 1)
delayed_flights_pd = delayed_flights.select("CRS_DEP_TIME").toPandas()
delayed_flights_pd['hour'] = delayed_flights_pd['CRS_DEP_TIME'] // 100
plt.figure(figsize=(12, 6))
plt.hist(delayed_flights_pd['hour'], bins=range(0, 25), edgecolor='black', color='salmon', align='left')
plt.title("Histogram of Delayed Flights by Scheduled Departure Hour")
plt.xlabel("Scheduled Departure Hour")
plt.ylabel("Number of Delayed Flights")
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

# for delayed flights, plotting a stacked bar chart of the number of flights at each time of day, broken down by airline
delayed_flights = join_data_5y_df.filter(F.col("DEP_DEL15") == 1)
delayed_flights_pd = delayed_flights.select("CRS_DEP_TIME", "OP_CARRIER").toPandas()
delayed_flights_pd['hour'] = delayed_flights_pd['CRS_DEP_TIME'] // 100
pivot_df = delayed_flights_pd.pivot_table(index='hour', columns='OP_CARRIER', aggfunc='size', fill_value=0)
pivot_df = pivot_df.sort_index()
plt.figure(figsize=(14, 8))
bottom = None
for carrier in pivot_df.columns:
    plt.bar(pivot_df.index, pivot_df[carrier], bottom=bottom, label=carrier)
    if bottom is None:
        bottom = pivot_df[carrier].copy()
    else:
        bottom += pivot_df[carrier]
plt.title("Stacked Bar Chart of Delayed Flights by Scheduled Departure Hour (by Airline)")
plt.xlabel("Scheduled Departure Hour")
plt.ylabel("Number of Delayed Flights")
plt.xticks(range(0, 24))
plt.legend(title="Airline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# calculate delay rate for each airline as delayed flights for a given day divided by flight volume for a given day, take the average daily delay rate, and then plot it as a bar chart over date
daily_flights = join_data_5y_df.groupBy("FL_DATE", "OP_CARRIER").count().withColumnRenamed("count", "total_flights")
daily_delays = join_data_5y_df.filter(F.col("DEP_DEL15") == 1).groupBy("FL_DATE", "OP_CARRIER").count().withColumnRenamed("count", "delayed_flights")
delay_rate_df = daily_flights.join(daily_delays, ["FL_DATE", "OP_CARRIER"], "left").fillna(0, subset=["delayed_flights"])
delay_rate_df = delay_rate_df.withColumn("delay_rate", F.col("delayed_flights") / F.col("total_flights"))
delay_rate_pd = delay_rate_df.groupBy("FL_DATE").agg(F.mean("delay_rate").alias("avg_delay_rate")).orderBy("FL_DATE").toPandas()
delay_rate_pd['FL_DATE'] = pd.to_datetime(delay_rate_pd['FL_DATE'])
plt.figure(figsize=(16, 6))
plt.bar(delay_rate_pd['FL_DATE'], delay_rate_pd['avg_delay_rate'], color='darkred', width=1)
plt.title("Bar Chart of Average Daily Delay Rate Across Airlines Over Time", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Average Delay Rate", fontsize=12)
plt.tight_layout()
plt.show()

# now separated out by carrier, plot as a stacked bar chart
delay_rate_pd = delay_rate_df.orderBy("FL_DATE").toPandas()
delay_rate_pd['FL_DATE'] = pd.to_datetime(delay_rate_pd['FL_DATE'])
pivot_df = delay_rate_pd.pivot(index='FL_DATE', columns='OP_CARRIER', values='delay_rate').fillna(0)
pivot_df = pivot_df.sort_index()
plt.figure(figsize=(16, 8))
bottom = None
for carrier in pivot_df.columns:
    plt.bar(pivot_df.index, pivot_df[carrier], bottom=bottom, label=carrier, width=1)
    if bottom is None:
        bottom = pivot_df[carrier].copy()
    else:
        bottom += pivot_df[carrier]
plt.title("Stacked Bar Chart of Daily Delay Rate by Airline Carrier Over Time", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Delay Rate", fontsize=12)
plt.legend(title="Airline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [0]:
# just do correlation heatmap for custom joined data
# correlation matrix and heat map of numeric variables
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# ['DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY' (fillna), 'WEATHER_DELAY' (fillna), 'NAS_DELAY' (fillna), 'SECURITY_DELAY' (fillna), 'LATE_AIRCRAFT_DELAY' (fillna), 'HourlyDryBulbTemperature', 'HourlyDewPointTemperature', 'HourlyRelativeHumidity', 'HourlyAltimeterSetting', 'HourlyVisibility', 'HourlyStationPressure', 'HourlyWetBulbTemperature', 'HourlyPrecipitation', 'HourlyCloudCoverage', 'HourlyCloudElevation', 'HourlyWindSpeed']
cols_to_keep = ['DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'HourlyDryBulbTemperature', 'HourlyDewPointTemperature', 'HourlyRelativeHumidity', 'HourlyAltimeterSetting', 'HourlyVisibility', 'HourlyStationPressure', 'HourlyWetBulbTemperature', 'HourlyPrecipitation', 'HourlyCloudCoverage', 'HourlyCloudElevation', 'HourlyWindSpeed']

# fill na for certain columns
join_data_5y_df = join_data_5y_df.fillna(0, subset=['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

assembler = VectorAssembler(inputCols=cols_to_keep, outputCol="features", handleInvalid="skip")
df_numeric = assembler.transform(join_data_5y_df.select(cols_to_keep)).select("features")
corr_matrix_row = Correlation.corr(df_numeric, "features", "pearson").collect()[0][0]
corr_matrix = corr_matrix_row.toArray().reshape(len(cols_to_keep), len(cols_to_keep))
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", xticklabels=cols_to_keep, yticklabels=cols_to_keep)
plt.title("Correlation Matrix (Numeric Features)")
plt.show()


# Markdown
extra EDA:
- correlation of different features (heatmap)
- delay propagation throughout the day (delay rate by hour of day) - broken down by airport or by carrier
- delay rate vs flight volume (by airport and by carrier)

notes from Vini:
- 19 min, cut to 14 min max
- we didn't do classification just yet, so be careful when including
    - we need to discuss the metrics here as well (eval each model?, eval on the whole?); how do we eval our final model that we use is a good one?
- EDA
    - need to explicitly specify where we are getting our data from (add link for data sources) - call it Data Sources
    - make number of nulls a table rather than a chart
    - no overlapping graphs, can split into multiple slides and say the same amount
    - be clear on the time frame (3 months, 6 months) - just specify
    - for custom join, explain that we are not dropping features until we do the custom join since then we can do the imputation
- custom join
    - make clear how many columns survive (have few enough nulls) after custom join 
- baseline model
    - how do we handle imbalance on the second model? maybe we'll want to just have 15-min delay and 30+
    - why are we using gap? so val can't learn from train
    - change our outcome to be log(1+departure delay)
- discuss feature engineering before we talk about model
- training results
    - can we put arrows in percentage instead of absolute numbers?
    - make sure we are making our decisions on at least 1 year of data
    - maybe holidays can be another feature?
- do error analysis once we're sure we have the best model (should be our ending)
- put pipeline much earlier in the presentation